In [18]:
import torch
import math
import json
import layers

In [19]:
torch.set_default_dtype(torch.float32)
if torch.cuda.is_available():
     torch.set_default_device(0)
     print("Running on the GPU")
else:
     print("Running on the CPU")

Running on the GPU


In [20]:
with open("bee20script.txt", "r") as file:
    data = file.read()
    
words = set(data.split(" "))
word_dict = {}
unique_words = len(words)
for i, word in enumerate(words):
    one_hot = torch.zeros(unique_words)
    one_hot[i] = 1
    word_dict[word] = one_hot


In [21]:
one_hot_encoded = []
for word in data.split(" "):
    one_hot_encoded.append(word_dict[word])

In [22]:
token_count = len(word_dict)
token_count

3169

In [23]:
d_model = 512
h = 4
n = 6
d_k = int(d_model / h)
d_v = d_k
attention_scaling = 1/(math.sqrt(d_k))
d_f_f = 4*d_model
num_tokens = 5

In [24]:
input_ = torch.column_stack(one_hot_encoded[0:5])
input_.size()

torch.Size([3169, 5])

In [25]:
def positional_encoding(num_tokens, d_model):
    encoding = torch.zeros(d_model, num_tokens)
    for pos in range(num_tokens):
        for i in range(0,d_model,2):
            encoding[i, pos] = math.sin(pos/(10000 ** ((2 * i) / d_model)))
            encoding[i + 1, pos] = math.cos(pos/(10000 ** ((2 * i) / d_model)))
    return encoding

In [27]:
main_input = layers.Input((token_count,num_tokens), train=False)
main_input.set(input_)

In [29]:
embedding_W = layers.Input((d_model,token_count), train=True)
embedding_W.set(torch.randn(d_model, token_count , requires_grad=True))
#From what I understand the embededing does not have a bias, my library has a bias required so I just have it set to 0
placeholder_bias = layers.Input((d_model,1), train=False)
placeholder_bias.set(torch.zeros(d_model,1))
embedding_layer = layers.Linear(main_input, embedding_W, placeholder_bias)